In [1]:
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from selenium.webdriver.common.keys import Keys

driver = uc.Chrome()

driver.maximize_window()
url = 'https://artrepreneur.com/search/artists?page=1&commercial=false&readyMade=false&searchDistanceMiles=20&'
driver.get(url)
wait = WebDriverWait(driver, 10)


In [2]:
page = 1
while True:
    
    page_url = f'https://artrepreneur.com/search/artists?page={page}&commercial=false&readyMade=false&searchDistanceMiles=124300&'
    print(f"Scraping page: {page}")
    
    driver.get(page_url)
   
    try:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//ul[@class="pagination justify-content-center"]')))
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        next_button = soup.find('ul', class_='pagination justify-content-center').find_all('li')[-1].get('class')[-1]
        
        if next_button == 'disabled':
            break
       
    
        col_tags = soup.find_all('div', class_='card rounded-1 h-300')
        artists_url = []
        for j in range(1, len(col_tags) + 1):


            artist_element = driver.find_element(By.XPATH, f'//*[@id="relativeContainer"]/main/div[2]/div[1]/div/div[1]/div[2]/div[{j}]/div/div[1]/h5')
            driver.execute_script("arguments[0].scrollIntoView();", artist_element)
            time.sleep(1) 

            driver.execute_script("arguments[0].click();", artist_element)

            WebDriverWait(driver, 10).until(EC.url_changes(url))


            artist_url=driver.current_url
            print(f"Artist URL: {artist_url}")
            driver.get(artist_url)
            driver.implicitly_wait(10)
            WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//a[@class="border-0 nav-link active"]'))
            ) 

            html_artist = driver.page_source
            soup_artist=BeautifulSoup(html_artist,'html.parser')

            follow_col_tags=soup_artist.find('div',class_='bg-lightGrey border-lightGrey d-flex p-3').find_all('div',class_='col')
            for follow_col_tag in follow_col_tags:
                if follow_col_tag.find('div', class_='fw-semiBold').text=='Followers':
                    followers=follow_col_tag.find('strong').text
                    print(followers," followers")
                if follow_col_tag.find('div', class_='fw-semiBold').text=='Following':
                    following=follow_col_tag.find('strong').text
                    print(following," following")

            if soup_artist.find('div', class_='fw-bold mt-2'):
                if soup_artist.find('div', class_='fw-bold mt-2').text == 'Bio':
                    bio_element=soup_artist.find('div', class_='fw-bold mt-2')
                    
                elif soup_artist.find('div', class_='desc-inner'):
                    artist_bio = soup_artist.find('div', class_='desc-inner').text
                       
                else:
                    artist_bio = None
                        #print(artist_bio)
                time.sleep(5)
            try:
                artist_name = soup_artist.find('h1',class_ = 'bio-name mt-3 mb-1').text
                print(artist_name)
            except:
                artist_name = None


            try:
                artist_nationality = soup_artist.find('div',class_='profile-header__avatar-social').find('div',class_='font-12 text-center').text
                print(artist_nationality)
            except:
                artist_nationality = None
                time.sleep(5)
            try:

                WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//span[@class="icon-search"]'))
                ) 
                driver.find_element('xpath', '//span[@class="icon-search"]').click()
                WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//nav[@aria-label="Page navigation"]'))
                )
                while True:
                        html_asset_page = driver.page_source
                        soup_asset_page = BeautifulSoup(html_asset_page, 'html.parser')
                        asset_div_tags = soup_asset_page.find('div', class_='react-photo-album react-photo-album--masonry').find_all('div', class_='subheadline1 text-break flex-1')

                        for asset_div_tag in asset_div_tags:
                            asset_url = 'https://artrepreneur.com' + asset_div_tag.find('a').get('href')
                            driver.get(asset_url)
                            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//span[@class="me-1"]')))
                            html_asset = driver.page_source
                            soup_asset = BeautifulSoup(html_asset, 'html.parser')

                


                        asset_tags=[]
                        try:
                            
                            span_tags=soup_asset.find('div',string='Search Terms').find_next_sibling().find('div',class_='h2-artwork').find_all('span')
                            for span_tag in span_tags:
                                tags=span_tag.text
                                asset_tags.append(tags)
                        except:
                            pass
                        try:
                            asset_title = soup_asset.find('div',class_='artwork-title my-2 text-break').text
                            print("asset title: ",asset_title)
                        except:
                            asset_title=None

                        try:  
                            seller_name = soup_asset.find('div',class_='font-15 text-break d-block d-sm-flex').find('a').text
                            print("seller: ",seller_name)
                        except:
                            seller_name=None
                        try: 
                            asset_description = soup_asset.find('div',class_='desc-inner').find('p').text
                            print("aaset description: ",asset_description)
                        except:
                            asset_description=None
                        try:
                            asset_created_date=soup_asset.find('div',string='Year Created').find_next_sibling('div').text
                            print("created: ",asset_created_date)
                        except:
                            asset_created_date=None
                        try:
                            asset_location=soup_asset.find('div',string='Location').find_next_sibling(string=True)
                            print("location: ",asset_location)
                        except:
                            asset_location=None

                        try:
                            asset_price_range=soup_asset.find('div',class_='col mx-auto').find('div').text
                            print(asset_title,"'s",asset_price_range)
                        except:
                            asset_price_range=None


                        driver.back()
                        time.sleep(2)
                    
                        driver.get(page_url)

            except:
                pass
                driver.get(page_url)

        page+=1
    except:
        break

Scraping page: 1
Artist URL: https://artrepreneur.com/p/Myriam-Vejus
0  followers
0  following
Myriam VÉJUS
Lyon, France
seller:  Myriam VÉJUS
aaset description:  Abrasion AB174, Guadeloupe, FWI.
created:  2024
location:  Lyon, France
Artist URL: https://artrepreneur.com/p/dL5nGHz7E3eekpnst
0  followers
0  following
Alexandra Bloodworth
Swansea, UK
seller:  Alexandra Bloodworth
aaset description:  “I love New York! ...but it's not always about the destination, in this case it's about the journey!”
created:  2024
Artist URL: https://artrepreneur.com/p/jonmann
0  followers
0  following
Jon Mann
USA
seller:  Jon Mann
aaset description:  This is one in a series of nude nature drawings I did on mat board with the intention of selling on the street. It has a big blue wave and a faceless woman posing in what could be a very precarious situation.
created:  2024
Artist URL: https://artrepreneur.com/p/reginae
0  followers
0  following
Regina Maria Erikson
Virginia, USA
seller:  Regina Maria Erik

Artist URL: https://artrepreneur.com/p/kurtpalmquistart
0  followers
0  following
Kurt Palmquist
Bozeman, MT, USA
seller:  Kurt Palmquist
aaset description:  "Strata" is a modern geometric sculpture that captures both the eye and the imagination through its innovative design and vibrant color palette. Measuring at a striking size that allows it to command attention in any space, this piece is constructed from interlocking, overlapping shapes that form a dynamic and visually engaging arrangement. The dominant color is a vivid orange, which stands out beautifully against the contrasting darker hues of black and gray used in the background layers.
created:  2020
None 's $3,500.00
Artist URL: https://artrepreneur.com/p/AndreaBruns
0  followers
0  following
Andrea Bruns
Vancouver, BC, Canada
seller:  Andrea Bruns
aaset description:  "Eternal Horizon," a serene and minimalist depiction of an ocean scene, captivates with its smooth, horizontal lines and calming colour palette. The hues transi